In [ ]:
import os
import yaml
import pandas as pd
from pyathena import connect
from pyathena.util import as_pandas
from sqlalchemy import *

%matplotlib inline
%load_ext autoreload

In [ ]:
pg_cred = yaml.load(open("../conf/local/credentials.yml"), Loader=yaml.FullLoader)

In [ ]:
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(pg_cred["pg_user"], pg_cred["pg_pass"], pg_cred["pg_host"], 5432, "iefp")

# The return value of create_engine() is our connection object
con = create_engine(url, client_encoding='utf8')

# We then bind the connection to MetaData()
meta = MetaData(bind=con, reflect=True)

In [ ]:
sql = """ 
select * from intervencoes
limit 10
;"""

intervencoes = pd.read_sql(sql, con)

In [ ]:
intervencoes.iloc[1:10,30:45]

In [ ]:
print(intervencoes.columns.values)

In [ ]:
sql = """ 
select distinct(f_dcurso) from intervencoes
;"""
dcurso = pd.read_sql(sql, con)

In [ ]:
dcurso.shape

In [ ]:
sql = """ 
select f_dcurso, count(f_dcurso) as count from intervencoes
group by f_dcurso
order by count DESC
;"""
dcurso_count = pd.read_sql(sql, con)

In [ ]:
dcurso_count.shape

In [ ]:
dcurso_count.head(10)

In [ ]:
dcurso_count.tail(10)

In [ ]:
dcurso_count[dcurso_count.count < 50]

In [ ]:
sql = """ 
select distinct(f_dmod_form) from intervencoes
;"""
dmod_form = pd.read_sql(sql, con)

In [ ]:
dmod_form.shape

In [ ]:
dmod_form.head()

In [ ]:
sql = """ 
select f_dmod_form, count(f_dmod_form) as count from intervencoes
group by f_dmod_form
order by count DESC
;"""
dmod_form_count = pd.read_sql(sql, con)

In [ ]:
dmod_form_count.shape

In [ ]:
dmod_form_count.loc[0:10,:]

## Insert intervencoes table using yaml file

In [ ]:
int_cols = yaml.load(open("../conf/base/sigae_columns.yml"), Loader=yaml.FullLoader)["intervencoes"]
table = 'intervencoes'

In [ ]:
sql = """
select {}
from {}
limit 10000
""".format(', '.join(int_cols), "intervencoes")

df = pd.read_sql(sql, con)

In [ ]:
df.head()

## Create function to clean up string columns

In [ ]:
# find string cols
df.dtypes

In [ ]:
# 1. change to lower case

In [ ]:
df_copy = df

In [ ]:
df_copy.f_dcurso.str.lower()

In [ ]:
# 2. remove characters

In [ ]:
df_copy.f_dcurso = df_copy.f_dcurso.str.replace('(|)|-', "", regex = True)
df_copy.f_dcurso

In [ ]:
def clean_string(column):
    "function to make string column lower case and remove characters defined in list"
    column = column.str.lower()
    characters= ['(',')','-','+','  ']
    for char in characters:
        column = column.str.replace(char, "", case = False, regex = False)
    
    return column

In [ ]:
df_copy = df

In [ ]:
# Check columns to see if we need more characters added to list 
cleaned = clean_string(df_copy.f_dcurso)
cleaned.drop_duplicates().to_list()

In [ ]:
df_copy.head()

In [ ]:
df_copy.dtypes

## Change the resultado_intervencao to time and then take out time

In [ ]:
df_copy["data_resultado"].astype("str").head()

In [ ]:
test = df_copy["data_resultado"].astype("str")
test.head()

In [ ]:
from datetime import datetime
datetime_object = datetime.strptime("2013-05-17 13:02:48","%Y-%m-%d %H:%M:%S")
datetime_object

In [ ]:
print(datetime_object.date())

In [ ]:
df_copy.data_intervencao.head()

In [ ]:
def object_to_date(column, format):
    """converts a object column to a string, and then to a datetime, and then removes the time"""
    column = column.astype("int64").astype("str")
    column = pd.to_datetime(column, errors='coerce', format = format)
    column = column.dt.date
    return column

In [ ]:
df_copy = df
from datetime import datetime
date_output = object_to_date(df_copy.data_resultado,"%Y-%m-%d %H:%M:%S")
date_output.head(5)

In [ ]:
# Test on other pedidos dates:

In [ ]:
sql = """
select *
from pedidos
limit 10000
"""

ped = pd.read_sql(sql, con)

In [ ]:
new = ped.ano_mes.astype("int64").astype("str")
new

In [ ]:
from datetime import datetime
date_output = object_to_date(ped.ano_mes, "%Y%m")
date_output.head(50)